<a href="https://colab.research.google.com/github/dongchanlim/Python/blob/master/Indeed_Employment_Data_Engineering_(Web_Scraping).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages
import pandas as pd
import numpy as np
import requests
import re
import string
from datetime import date
from bs4 import BeautifulSoup

In [ ]:
# object generation (Area, Country)
class Area:
  def __init__(self, area):
    self.area = area

class Country(Area):
  def __init__(self, area, country, indeed_url):
    super().__init__(area)
    self.country = country
    self.indeed_url = indeed_url


In [ ]:
# get  a list of country name and url
indeed_countries = []

# request url 
country_indeed = requests.get("https://www.indeed.com/worldwide")

# convert source_code (text) into python object
country_soup = BeautifulSoup(country_indeed.text,  "html.parser")

# get list name and url of country
country_list = country_soup.find_all('li', {'class': 'countryItem'})

# save country objects into indeed_countires list
for li in country_list:
  #print(li.text)
  #try:
  #  print(li.a['href'])
  #except:
  #  print("https://indeed.com")
  name = li.text.replace("\n","")
  try:
    url = li.a['href']
  except:
    url = 'https://indeed.com'
  if name == "Argentina":
    area = "South America Area"
  c = Country(name, url, area)
  indeed_countries.append(c)

In [ ]:
# Validate the object and instance
print(indeed_countries[0].country)
print(indeed_countries[0].indeed_url)
print(indeed_countries[0].area)

https://ar.indeed.com/
South America Area
Argentina


In [ ]:
# get a PW certificate from PW website
pathway_cert_url = "https://www.byupathway.org/degrees"

html = requests.get(pathway_cert_url).text

pathway_soup = BeautifulSoup(html,"html.parser")

cert = pathway_soup.find_all('div', {"class":"ListLabeledLinks-items-item"})

cert_list = [i.a.span.text for i in cert][13:]

cert_list = [i.replace(' ','+') for i in cert_list]

print(cert_list)

len(cert_list)

['Administrative+Assistant', 'Advanced+Family+History+Research+', 'Advanced+Marriage+and+Family+Functioning', 'Agribusiness+Management', 'Auto+Services+Technology', 'Basic+Accounting', 'Business+Administration', 'Business+Analysis', 'Business+and+Leadership+Skills', 'Child+and+Family+Advocacy', 'Commercial+Fundamentals', 'Communication+Core', 'Communication+Fundamentals', 'Community+Health+Methods+&+Evaluation', 'Community+Health+Planning+&+Implementation', 'Computer+Support', 'Computer-Aided+Design+and+Drafting', 'Construction+Field+Supervision', 'Database', 'Entrepreneurship', 'Epidemiology', 'Family+History+Research', 'Graphic+Design+Fundamentals+', 'Hospitality+and+Tourism+Management', 'Human+Resource+Management', 'Human+Services', 'Marriage,+Family,+and+Human+Relations', 'Medical+Billing+and+Coding+Fundamentals', 'Occupational+Safety+and+Health', 'Parent+and+Family+Education', 'Project+Management+', 'Social+Media+Marketing', 'System+Administration', 'TEFL', 'Web+and+Computer+Progr

36

In [59]:
# date, certificate, and how many job are posted
record_date = date.today().strftime("%Y/%m/%d")
phil = "https://ph.indeed.com/"
job = "Basic+Accounting"


def get_result_num(country_website, job):
  url = requests.get(country_website + "jobs?q=" + job + "&explvl=entry_level&fromage=1")
  html = url.text
  soup = BeautifulSoup(html, "html.parser")
  # titles = soup.find_all('h2', {"class": "title"})
  # areas = soup.find_all('span', {"class": "location accessible-contrast-color-location"})
  try:
    result_num = re.findall('\d+', soup.find("div", {"id":"searchCountPages"}).text.replace(",",""))
    return int(result_num[1])
  except:
    return 0

result = get_result_num(phil, job)

print(job, "has" , result, "result")


{"date": record_date, 
 "job" : job, 
 "count" : result}

# next goal: assign the different country url to routine (Monday: Itary Indeed, Basic Accounting)


Basic+Accounting has 61 result


{'count': 61, 'date': '2021/03/09', 'job': 'Basic+Accounting'}

In [ ]:
Date = pd.DataFrame({'date': pd.Series(record_date)})
Date['key'] = 0

Keyword = pd.DataFrame({'search': cert_list})
Keyword['key'] = 0


df = Date.merge(Keyword, how = 'outer')


# df_us['website'] = "https://ph.indeed.com/"

In [72]:
def get_result_num_per_area(country_website, job):
  url = requests.get(country_website + "jobs?q=" + job + "&explvl=entry_level&fromage=1")
  html = url.text
  soup = BeautifulSoup(html, "html.parser")
  result_area = soup.findAll("span", {"class":"rbLabel"})
  area = [i.text.replace("\xa0(", "").replace(")","") for i in result_area]
  df = pd.DataFrame({'category': [area[i] for i in range(len(area)) if i % 2 == 0],
                     'count': [area[i] for i in range(len(area)) if i % 2 != 0]})
  
  return df

In [74]:
# get area and the number per area
get_result_num_per_area("https://indeed.com/", "Agribusiness+Management")

,category,count
0,Temporarily remote (COVID-19,1
1,Full-time,10
2,Internship,1
3,"New Iberia, LA",1
4,"Yadkinville, NC",1
5,Minnesota,1
6,"Black, AL",1
7,"Algona, IA",1
8,"Boise, ID",1
9,"Caledonia, MI",1


In [75]:
# English Used Country
df['us_result_num'] = df['search'].apply(lambda x: get_result_num("https://indeed.com/", x))

In [76]:
df['uk_result_num'] = df['search'].apply(lambda x: get_result_num("https://uk.indeed.com/", x))

In [77]:
df['ph_result_num'] = df['search'].apply(lambda x: get_result_num("https://ph.indeed.com/", x))

In [78]:
df

,date,key,search,ph_result_num,us_result_num,uk_result_num
0,2021/03/09,0,Administrative+Assistant,74,5060,415
1,2021/03/09,0,Advanced+Family+History+Research+,0,63,2
2,2021/03/09,0,Advanced+Marriage+and+Family+Functioning,0,2,0
3,2021/03/09,0,Agribusiness+Management,3,11,2
4,2021/03/09,0,Auto+Services+Technology,2,840,55
5,2021/03/09,0,Basic+Accounting,61,1187,77
6,2021/03/09,0,Business+Administration,241,2234,1627
7,2021/03/09,0,Business+Analysis,287,2695,1932
8,2021/03/09,0,Business+and+Leadership+Skills,170,7641,1727
9,2021/03/09,0,Child+and+Family+Advocacy,0,148,5


In [112]:
def get_top_cert(country):
  curr_date = date.today().strftime("%Y/%m/%d")
  if country == "ph":
    result = df.loc[(df['ph_result_num'].nlargest(5, keep = "all")) & (df['date'] == curr_date)]['search']
  elif country == "us":
    result = df.loc[(df['us_result_num'].nlargest(5, keep = "all")) & (df['date'] == curr_date)]['search']
  elif country == "uk":
    result = df.loc[(df['uk_result_num'].nlargest(5, keep = "all")) & (df['date'] == curr_date)]['search']
  return result


In [114]:
get_top_cert('ph')
#get_top_cert('us')
#get_top_cert('uk')

6     Business+Administration
7           Business+Analysis
15           Computer+Support
Name: search, dtype: object

In [ ]:
class search_result:
  
  def __init__(self, title, area, require):
    self.title = title
    self.area = area
    self.require = require
